# UnicornTest Data Analysis

## Imports and Presets

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider, FloatSlider

### Define to Project-Root Directory

In [3]:
PROJECT_ROOT = 'bio-feedback'

### Navigate to Project-Root Directory

In [4]:
currend_wd = os.getcwd()
project_root_abs = os.path.join(currend_wd[:currend_wd.find(PROJECT_ROOT)], PROJECT_ROOT)
os.chdir(project_root_abs)

### Import Project-Specific Libraries

In [5]:
from biofb.io import SessionDatabase as SessionDB

## Load DB-Files

In [6]:
db = SessionDB.load(filename='data/session/db-2021-01-29-bioplux-unicorn.yml')
db += SessionDB.load(filename='data/session/db-2021-02-19-bioplux-unicorn.yml')

[print(f'{s}: {s.setting}') for s in db.samples];

<Sample: Subject 882947 at 2021-01-29 15:26:38>: <Setting: Controlled hypnosis>
<Sample: Subject 165278 at 2021-01-29 16:11:27>: <Setting: Controlled hypnosis>
<Sample: Subject 925022 at 2021-01-29 16:59:30>: <Setting: Controlled hypnosis>
<Sample: Subject 882947 at 2021-02-19 15:20:23>: <Setting: Controlled hypnosis>
<Sample: Subject 165278 at 2021-02-19 16:09:44>: <Setting: Controlled hypnosis>
<Sample: Subject 746505 at 2021-02-19 16:52:19>: <Setting: Controlled hypnosis>
<Sample: Subject 925022 at 2021-02-19 17:32:45>: <Setting: Controlled hypnosis>


### Show Device-List

In [7]:
for s in db.samples:
    print(s)
    [print(f'device {i}: {d}, {d.description}') for i, d in enumerate(s.setup.devices)]
    print()

<Sample: Subject 882947 at 2021-01-29 15:26:38>
device 0: <Bioplux: bioplux>, biosignalsplux 8-channel hub, 7-channel acquisition
device 1: <Unicorn: unicorn>, g.tec Unicorn Suit Hybrid Black, channels: 8-EEG ACC-XYZ GYR-XYZ BAT VALID

<Sample: Subject 165278 at 2021-01-29 16:11:27>
device 0: <Bioplux: bioplux>, biosignalsplux 8-channel hub, 7-channel acquisition
device 1: <Unicorn: unicorn>, g.tec Unicorn Suit Hybrid Black, channels: 8-EEG ACC-XYZ GYR-XYZ BAT VALID

<Sample: Subject 925022 at 2021-01-29 16:59:30>
device 0: <Bioplux: bioplux>, biosignalsplux 8-channel hub, 7-channel acquisition
device 1: <Unicorn: unicorn>, g.tec Unicorn Suit Hybrid Black, channels: 8-EEG ACC-XYZ GYR-XYZ BAT VALID

<Sample: Subject 882947 at 2021-02-19 15:20:23>
device 0: <Bioplux: bioplux>, biosignalsplux 8-channel hub, 7-channel acquisition
device 1: <Unicorn: unicorn>, g.tec Unicorn Suit Hybrid Black, channels: 8-EEG ACC-XYZ GYR-XYZ BAT VALID

<Sample: Subject 165278 at 2021-02-19 16:09:44>
device 0

## Visualize Data (Interactive)

In [8]:
# pick a data-slice restriction (optional)
data_slice = slice(0,-1,1)

In [9]:
@interact
def view_bioplux(
    sample=[i for i in range(len(db.samples))],
    label_by=['name', 'label'],
    show_sftf=False,
    cmap='magma',
    update=False,
):

    try:
        s = db.samples[int(sample)]
        print(s)
        print()
        [print(c) for c in s.comments]

        s_time = s.time[0]
        s_data = s.data[0]

        f, all_axes = plt.subplots(sum([len(d.channels) for d in s.setup.devices]), 1, figsize=(15,30), sharex=True)
        
        ci, cj = 0, 0
        for i, device in enumerate(s.setup.devices):
            cj = ci + len(device.channels)
            axes = all_axes[ci:cj]
            ci = cj            

            if show_sftf:

                for ax, channel in zip(axes, device.channels):
                    sr = channel.sampling_rate
                    data = channel.data
                    Pxx, freqs, bins, im = ax.specgram(data[data_slice], NFFT=sr, Fs=sr, noverlap=sr//2, scale='dB', cmap=cmap)
                    ax.set_ylabel(getattr(channel, label_by))
                    ax.set_ylim([0,60])
                axes[-1].set_xlabel('Time')
            else:
                device.plot(axes=axes, label_by=label_by)
                axes[-1].set_xlim([s_time[data_slice, 0][0], s_time[data_slice, 0][-1]])          
            plt.tight_layout

    except Exception as ex:
        print('something went wrong:', ex)
        raise ex


interactive(children=(Dropdown(description='sample', options=(0, 1, 2, 3, 4, 5, 6), value=0), Dropdown(descrip…